In [314]:
import pandas as pd
import numpy as np
from shapely.geometry import Point
import geopandas as gpd
import requests

In [315]:
def to_mercator(lat, long):
    # transform latitude/longitude data in degrees to pseudo-mercator coordinates in metres
    c = gpd.GeoSeries([Point(lat, long)], crs=4326)
    c = c.to_crs(3857)
    return c
def distance_meters(mercator_start, mercator_finish):
    # return the distance in metres between to latitude/longitude pair point in degrees (i.e.: 40.392436 / -3.6994487)
    return mercator_start.distance(mercator_finish)

In [316]:
def monuments():
    response = requests.get('https://datos.madrid.es/egob/catalogo/208844-0-monumentos-edificios.json')
    resultado = response.json()
    monuments = pd.json_normalize(resultado['@graph'])
    monuments["Type of place"]= "Edificios de carácter monumental"
    monuments = monuments.rename(columns={"title": "Place of interest", "address.street-address": "Place address", "location.longitude": "Long_start", "location.latitude": "Lat_start"}, errors="raise")
    monuments_clean = monuments[["Place of interest","Type of place","Place address","Long_start","Lat_start"]]
    monuments_clean['Lat_start'] = pd.to_numeric(monuments_clean['Lat_start'],errors = 'coerce')
    monuments_clean['Long_start'] = pd.to_numeric(monuments_clean['Long_start'],errors = 'coerce')
    #monuments['NUM']= monuments['NUM'].astype(str).replace("nan"," SN")
    #monuments["Dirección"] = monuments["CLASE-VIAL"] + " " + monuments["NOMBRE-VIA"] + " Nº" + monuments["NUM"]
    return monuments_clean

def stations():
    stations = pd.read_json("../data/bicimad_stations.json", orient='records')
    geometry_coordinates = stations["geometry_coordinates"].str.split(expand=True)
    geometry_coordinates.columns = ['LONGITUD', 'LATITUD']
    geometry_coordinates['LONGITUD'] = geometry_coordinates['LONGITUD'].str.replace("[","")
    geometry_coordinates['LONGITUD'] = geometry_coordinates['LONGITUD'].str.replace(",","")
    geometry_coordinates['LATITUD'] = geometry_coordinates['LATITUD'].str.replace("]","")
    stations = pd.concat([stations, geometry_coordinates], axis=1)
    stations_clean = stations.rename(columns={"name": "BiciMAD station", "address": "Station location", "LONGITUD": "Long_finish", "LATITUD": "Lat_finish"}, errors="raise")
    stations_clean = stations_clean[["BiciMAD station","Station location","Long_finish","Lat_finish"]]
    stations_clean['Lat_finish'] = pd.to_numeric(stations_clean['Lat_finish'],errors = 'coerce')
    stations_clean['Long_finish'] = pd.to_numeric(stations_clean['Long_finish'],errors = 'coerce')
    return stations_clean

/Users/albertofernandez/miniconda3/envs/proy-env/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/Users/albertofernandez/miniconda3/envs/proy-env/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/albertofernandez/miniconda3/envs/proy-env/lib/python3.7/site-packages/ipykernel_launcher.py:16: FutureWarning: The default value of regex will change from True to False in a future version. In addi

In [317]:
def mercator_1():
    monuments_clean["Mercator_start"] = monuments_clean.apply(lambda x: to_mercator(x['Lat_start'],x['Long_start']),axis=1)
    return monuments_clean
def mercator_2():
    stations_clean["Mercator_finish"] = stations_clean.apply(lambda x: to_mercator(x['Lat_finish'],x['Long_finish']),axis=1)
    return stations_clean

/Users/albertofernandez/miniconda3/envs/proy-env/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [318]:
def merge():
    df_sumary = pd.merge(monuments_clean,stations_clean, how="cross")
    return df_sumary

In [319]:
def apply_distance_meters():
    df_sumary["Distance"] = df_sumary.apply(lambda x: distance_meters(x['Mercator_start'],x['Mercator_finish']),axis=1)
    return df_sumary


In [320]:
df_sumary

,Place of interest,Type of place,Place address,Long_start,Lat_start,Mercator_start,BiciMAD station,Station location,Long_finish,Lat_finish,Mercator_finish,Distance
0,Almacenes Rodríguez,Edificios de carácter monumental,Calle CABALLERO DE GRACIA 3,-3.700924,40.419716,POINT (4499502.203189674 -412271.7922079021),Puerta del Sol A,Puerta del Sol nº 1,-3.701834,40.417214,POINT (4499223.648366921 -412373.2851572837),296.468562
1,Almacenes Rodríguez,Edificios de carácter monumental,Calle CABALLERO DE GRACIA 3,-3.700924,40.419716,POINT (4499502.203189674 -412271.7922079021),Puerta del Sol B,Puerta del Sol nº 1,-3.701603,40.417313,POINT (4499234.670122574 -412347.498528782),278.038467
2,Almacenes Rodríguez,Edificios de carácter monumental,Calle CABALLERO DE GRACIA 3,-3.700924,40.419716,POINT (4499502.203189674 -412271.7922079021),Miguel Moya,Calle Miguel Moya nº 1,-3.705842,40.420589,POINT (4499599.340516399 -412820.3206106317),557.062895
3,Almacenes Rodríguez,Edificios de carácter monumental,Calle CABALLERO DE GRACIA 3,-3.700924,40.419716,POINT (4499502.203189674 -412271.7922079021),Plaza Conde Suchil,Plaza del Conde del Valle de Súchil nº 3,-3.706917,40.430294,POINT (4500679.707306497 -412940.3068148262),1354.041257
4,Almacenes Rodríguez,Edificios de carácter monumental,Calle CABALLERO DE GRACIA 3,-3.700924,40.419716,POINT (4499502.203189674 -412271.7922079021),Malasaña,Calle Manuela Malasaña nº 5,-3.702587,40.428552,POINT (4500485.866677178 -412457.3286497012),1001.008306
...,...,...,...,...,...,...,...,...,...,...,...,...
55963,Zoo Aquarium de Madrid,Edificios de carácter monumental,COMPLEJO ZOOLOGICO DE LA CASA CAMPO,-3.761360,40.411760,POINT (4498616.578385554 -419013.7871770847),Ciudad Universitaria 1,Avenida de la Complutense (Metro Ciudad Univer...,-3.726990,40.443750,POINT (4502177.655770458 -415179.5220676512),5232.863563
55964,Zoo Aquarium de Madrid,Edificios de carácter monumental,COMPLEJO ZOOLOGICO DE LA CASA CAMPO,-3.761360,40.411760,POINT (4498616.578385554 -419013.7871770847),Ciudad Universitaria 2,Avenida de la Complutense (Metro Ciudad Univer...,-3.726930,40.443420,POINT (4502140.920338497 -415172.8287427756),5212.863694
55965,Zoo Aquarium de Madrid,Edificios de carácter monumental,COMPLEJO ZOOLOGICO DE LA CASA CAMPO,-3.761360,40.411760,POINT (4498616.578385554 -419013.7871770847),Facultad Biología,Calle José Antonio Novais frente al nº 12,-3.727295,40.448332,POINT (4502687.743941171 -415213.4906984274),5569.258686
55966,Zoo Aquarium de Madrid,Edificios de carácter monumental,COMPLEJO ZOOLOGICO DE LA CASA CAMPO,-3.761360,40.411760,POINT (4498616.578385554 -419013.7871770847),Facultad Derecho,Avenida Complutense nº 23,-3.729370,40.451090,POINT (4502994.74083288 -415445.024322674),5648.395766


In [388]:
def minimum():
    df_filter = df_sumary[df_sumary["Place of interest"] == input('Pon el lugar de interés: ')]
    pre_minimum = df_filter[df_filter['Distance'] == df_filter['Distance'].min()]
    minimum = pre_minimum[["Place of interest","Type of place","Place address","BiciMAD station","Station location","Distance"]]
    return minimum


In [389]:
minimum()

Pon el lugar de interés: Banco Matritense


,Place of interest,Type of place,Place address,BiciMAD station,Station location,Distance
1338,Banco Matritense,Edificios de carácter monumental,Calle GRAN VIA 22,Plaza de Pedro Zerolo,Plaza de Pedro Zerolo nº 1,95.222046


In [431]:
#def all_places():
    #df_minimum = pd.DataFrame(columns=["Place of interest","Type of place","Place address","BiciMAD station","Station location"])
    #for place in monuments_clean["Place of interest"]:
        #df_filter2 = df_sumary[df_sumary["Place of interest"] == place]
        #df_places =  df_filter2[df_filter2['Distance'] == df_filter2['Distance'].min()]
        #df_places_clear = df_places[["Place of interest","Type of place","Place address","BiciMAD station","Station location"]]
        #df_minimum = df_minimum.append(df_places_clear)
    #return df_minimum

    

In [451]:
def all_places():
    return df_sumary.sort_values(by = "Distance", ascending = True).groupby('Place of interest')['Type of place','Place address','BiciMAD station', 'Station location','Distance'].nth(0).drop(["Distance"], axis = "columns")


In [452]:
all_places()

/Users/albertofernandez/miniconda3/envs/proy-env/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  


,Type of place,Place address,BiciMAD station,Station location
Place of interest,,,,
Almacenes Rodríguez,Edificios de carácter monumental,Calle CABALLERO DE GRACIA 3,Plaza de Pedro Zerolo,Plaza de Pedro Zerolo nº 1
Antigua Fábrica Martini &amp; Rossi,Edificios de carácter monumental,AVENIDA ARAGON 328,Gutierre de Cetina,Calle Gutierre de Cetina nº 77
Ateneo de Madrid,Edificios de carácter monumental,Calle PRADO 21,Plaza de Santa Ana,Plaza de Santa Ana nº 10
Banco Bilbao Vizcaya Argentaria,Edificios de carácter monumental,Calle ALCALÁ 16,Sevilla,Calle Alcalá nº 27
Banco Matritense,Edificios de carácter monumental,Calle GRAN VIA 22,Plaza de Pedro Zerolo,Plaza de Pedro Zerolo nº 1
...,...,...,...,...
Viviendas y oficinas para Seguros La Estrella,Edificios de carácter monumental,Calle CABALLERO DE GRACIA 15,Sevilla,Calle Alcalá nº 27
Viviendas y oficinas para el vizconde de Escoriaza,Edificios de carácter monumental,Calle GRAN VIA 22B,Plaza de Pedro Zerolo,Plaza de Pedro Zerolo nº 1
Viviendas y oficinas para la Constructora Calpense,Edificios de carácter monumental,Calle CHINCHILLA,Tres Cruces,Calle Tres Cruces nº 7
